### Todo:
-setup ts neuralnet

-write GA


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
data = np.genfromtxt('two_spirals.dat')
seed = 42069 
rng = np.random.RandomState(seed)

In [3]:
train_split = 0.5
train_size = int(data.shape[0]*train_split)

data_train = data[0:train_size]
data_test = data[train_size:]

x_train = data_train[:, 0:2]
y_train = data_train[:, 2]
x_test = data_test[:, 0:2]
y_test = data_test[:, 2]


In [4]:
model = tf.keras.models.Sequential([
  #define network structure here
  tf.keras.layers.Dense(8, activation='tanh'),
  tf.keras.layers.Dense(2, activation='tanh')
])
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
5/5 [==============================] - 0s 1ms/step - loss: 4.2652 - accuracy: 0.5000
Epoch 2/5
5/5 [==============================] - 0s 1ms/step - loss: 4.2039 - accuracy: 0.4924
Epoch 3/5
5/5 [==============================] - 0s 1ms/step - loss: 4.1982 - accuracy: 0.4848
Epoch 4/5
5/5 [==============================] - 0s 1ms/step - loss: 4.1948 - accuracy: 0.4848
Epoch 5/5
5/5 [==============================] - 0s 2ms/step - loss: 4.1922 - accuracy: 0.4924
5/5 - 0s - loss: 4.1575 - accuracy: 0.4924


[4.157505512237549, 0.49242424964904785]

In [21]:
def ga(termination_accuracy = 1.0, pop_size = 20, mutation_prob = 0.01):
    population = (rng.uniform(low = 0.0, high = 1.0, size = (pop_size, 14)) > 0.5) * 1
    max_score = 0
    best_model = 0
    best_model_params = []
    generation = 0
    while max_score<termination_accuracy:
        print('Generation ' + str(generation))
        generation+=1
        scores = np.zeros(pop_size)
        #calculate fitness
        for i in np.arange(pop_size):
            score, model = genome_score(population[i], x_train, y_train, x_test, y_test)
            if score > max_score:
                max_score = score
                best_model = model
                best_model_params = population[i]
            scores[i] = score

        #selection
        population = roulette_selection(population, scores)

        #make sexy time
        i=0
        while i<pop_size:
            c1,c2 = crossover(population[i], population[i+1])
            population[i] = c1
            population[i+1] = c2
            i+=2

        #mutation
        population = mutation(population, mutation_prob)
    return best_model, best_model_params

In [22]:
model, params = ga()
print(model.evaluate(np.concatenate((x_test, np.sin(x_test)), axis=1), y_test))
print(params)

Generation 0
5/5 [==============================] - 0s 598us/step - loss: 0.0322 - accuracy: 1.0000
[0.032158687710762024, 1.0]
[1 0 1 1 0 1 0 1 0 0 0 1 0 1]


In [7]:
def roulette_selection(population, scores):
    N = population.shape[0]
    total_fitness = np.sum(scores)
    
    if total_fitness == 0: 
        relative_fitness = scores +1/N
    else:
        relative_fitness = scores / total_fitness
    
    indices = np.arange(N)
    selected_indices = rng.choice(indices, N, p=relative_fitness)
    return population[selected_indices]

In [8]:
def crossover(p1, p2):
    point = rng.randint(0, len(p1))
    child1 = np.concatenate((p1[0:point], p2[point:]))
    child2 = np.concatenate((p2[0:point], p1[point:]))
    return child1, child2

In [9]:
def mutation(population, mutation_prob):
    mutated_pop = np.copy(population)
    for i in np.arange(population.shape[0]):
        for j in np.arange(population.shape[1]):
            if rng.rand()<mutation_prob:
                mutated_pop[i,j] = abs(population[i][j]-1)
    return mutated_pop

In [10]:
a = np.array([[0,1,1,0,0,0,1,0,1], [1,0,1,1,1,0,0,1,0]])
print(mutation(a, 0.5))
print(a)

[[1 1 0 0 1 0 1 1 0]
 [1 0 0 0 0 0 0 1 0]]
[[0 1 1 0 0 0 1 0 1]
 [1 0 1 1 1 0 0 1 0]]


In [11]:
rng.choice([0,1,2], 20, p=[1,0,0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
def genome_score(genome, x_train, y_train, x_test, y_test, n_epochs = 400, learning_rate = 0.1):
    #Change the input layer from (x1, x2) to (x1, x2, sin(x1), sin(x2))
    x_train = np.concatenate((x_train, np.sin(x_train)), axis=1)
    x_test = np.concatenate((x_test, np.sin(x_test)), axis=1)
    decoded = decode_genome(genome)
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(4,)))
    for i in np.arange(decoded[0]):
        model.add(tf.keras.layers.Dense(decoded[i+1], activation="tanh"))
    
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
    model.fit(x_train, y_train, epochs = n_epochs, verbose=0)
    return model.evaluate(x_test, y_test)[1], model

In [13]:
def decode_genome(arr):
    num_layers = arr[0]*2 + arr[1] + 1
    first_layer = arr[2]*4 + arr[3]*2 + arr[4] + 1
    second_layer = arr[5]*4 + arr[6]*2 + arr[7] + 1
    third_layer = arr[8]*4 + arr[9]*2 + arr[10] + 1
    fourth_layer = arr[11]*4 + arr[12]*2 + arr[13] + 1
    return num_layers, first_layer, second_layer, third_layer, fourth_layer

In [14]:
x_train_a = np.concatenate((x_train, np.sin(x_train)), axis=1)
x_test_a = np.concatenate((x_test, np.sin(x_test)), axis=1)
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation="tanh"))
model.add(tf.keras.layers.Dense(8, activation="tanh"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=['accuracy'])
model.fit(x_train_a, y_train, epochs = 500)
model.evaluate(x_test_a, y_test)

Epoch 1/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7949 - accuracy: 0.4773
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7532 - accuracy: 0.4621
Epoch 3/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7251 - accuracy: 0.5000
Epoch 4/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7117 - accuracy: 0.5227
Epoch 5/500
5/5 [==============================] - 0s 997us/step - loss: 0.7039 - accuracy: 0.4773
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6971 - accuracy: 0.5000
Epoch 7/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6957 - accuracy: 0.4773
Epoch 8/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.4848
Epoch 9/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6880 - accuracy: 0.5303
Epoch 10/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6878 - accuracy: 0.5076
Epoch 11/500
5/5 [=========

5/5 [==============================] - 0s 1ms/step - loss: 0.6435 - accuracy: 0.5379
Epoch 83/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6374 - accuracy: 0.5758
Epoch 84/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6388 - accuracy: 0.5682
Epoch 85/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6342 - accuracy: 0.5530
Epoch 86/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6374 - accuracy: 0.6439
Epoch 87/500
5/5 [==============================] - 0s 997us/step - loss: 0.6366 - accuracy: 0.6288
Epoch 88/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6385 - accuracy: 0.6212
Epoch 89/500
5/5 [==============================] - 0s 1ms/step - loss: 0.6373 - accuracy: 0.6212
Epoch 90/500
5/5 [==============================] - 0s 997us/step - loss: 0.6288 - accuracy: 0.5530
Epoch 91/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6294 - accuracy: 0.5303
Epoch 92/500
5/5 [===========

5/5 [==============================] - 0s 1ms/step - loss: 0.5284 - accuracy: 0.7348
Epoch 165/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5306 - accuracy: 0.7045
Epoch 166/500
5/5 [==============================] - 0s 997us/step - loss: 0.5263 - accuracy: 0.7727
Epoch 167/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5233 - accuracy: 0.7576
Epoch 168/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5242 - accuracy: 0.6742
Epoch 169/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5238 - accuracy: 0.6667
Epoch 170/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5218 - accuracy: 0.6591
Epoch 171/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5194 - accuracy: 0.6591
Epoch 172/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5296 - accuracy: 0.7727
Epoch 173/500
5/5 [==============================] - 0s 1ms/step - loss: 0.5242 - accuracy: 0.7576
Epoch 174/500
5/5 [===

5/5 [==============================] - 0s 1ms/step - loss: 0.4530 - accuracy: 0.7803
Epoch 246/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.8258
Epoch 247/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4477 - accuracy: 0.8258
Epoch 248/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4490 - accuracy: 0.7955
Epoch 249/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4497 - accuracy: 0.8258
Epoch 250/500
5/5 [==============================] - 0s 1ms/step - loss: 0.4459 - accuracy: 0.8333
Epoch 251/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4469 - accuracy: 0.7879
Epoch 252/500
5/5 [==============================] - 0s 997us/step - loss: 0.4460 - accuracy: 0.8106
Epoch 253/500
5/5 [==============================] - 0s 1ms/step - loss: 0.4415 - accuracy: 0.7955
Epoch 254/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4415 - accuracy: 0.8182
Epoch 255/500
5/5 [===

5/5 [==============================] - 0s 1ms/step - loss: 0.3512 - accuracy: 0.8409
Epoch 328/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3348 - accuracy: 0.8939
Epoch 329/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3390 - accuracy: 0.8636
Epoch 330/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3337 - accuracy: 0.8712
Epoch 331/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3561 - accuracy: 0.8485
Epoch 332/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3494 - accuracy: 0.8788
Epoch 333/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3277 - accuracy: 0.8788
Epoch 334/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3284 - accuracy: 0.8939
Epoch 335/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3282 - accuracy: 0.9015
Epoch 336/500
5/5 [==============================] - 0s 1ms/step - loss: 0.3285 - accuracy: 0.9015
Epoch 337/500
5/5 [=====

5/5 [==============================] - 0s 997us/step - loss: 0.2594 - accuracy: 0.8636
Epoch 410/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2161 - accuracy: 0.9545
Epoch 411/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2343 - accuracy: 0.9318
Epoch 412/500
5/5 [==============================] - 0s 997us/step - loss: 0.2229 - accuracy: 0.9394
Epoch 413/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2143 - accuracy: 0.9242
Epoch 414/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2188 - accuracy: 0.9091
Epoch 415/500
5/5 [==============================] - 0s 997us/step - loss: 0.2145 - accuracy: 0.9242
Epoch 416/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2074 - accuracy: 0.9318
Epoch 417/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2559 - accuracy: 0.8864
Epoch 418/500
5/5 [==============================] - 0s 997us/step - loss: 0.2078 - accuracy: 0.9394
Epoch 419/500
5/

5/5 [==============================] - 0s 1ms/step - loss: 0.1372 - accuracy: 0.9773
Epoch 492/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1306 - accuracy: 0.9545
Epoch 493/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1394 - accuracy: 0.9394
Epoch 494/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1392 - accuracy: 0.9394
Epoch 495/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1459 - accuracy: 0.9621
Epoch 496/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1368 - accuracy: 0.9621
Epoch 497/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1356 - accuracy: 0.9621
Epoch 498/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1276 - accuracy: 0.9697
Epoch 499/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1456 - accuracy: 0.9318
Epoch 500/500
5/5 [==============================] - 0s 798us/step - loss: 0.1612 - accuracy: 0.9394


[0.16117316484451294, 0.939393937587738]

In [23]:
decode_genome([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

(3, 7, 6, 3, 6)

In [ ]:
[1 0 1 1 0 1 0 1 0 0 0 1 0 1]